# API Data Report                                           
### John Cobb - October 15, 2023

#### Hypothesis: I believe that newer/more recently created songs have higher energy than songs that are older.

##### Impact: The theoretical and statistical applications of this hypothesis would be very important in identifying the trend in which music might be heading. Say the hypothesis were proven true, going forward song writers and producers might want to continue raising the "energy" of the songs they create in order to fit in the current music environment. Understanding how time periods/eras affect energy in songs could be useful in finding and understanding what songs would be succesful at what times. It could also provide insight into what music would be effective marketing for reaching certain age levels. Although a lot could be inferred from the results of the hypothesis, in order to truly understand the full extent of the results it would be crucial in understanding what energy is and how Spotify defines it. 

##### "Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy" (Spotify for Developers).

The two endpoints I ended up using were the top 50 songs from the years 1969 and 2023 as well as their track features. I used these two end points because I felt 1969 was a huge year in music and used the current year as a basis of comparison. Ideally I would have used a range of years but I didn't want to overcomplicate what I was trying to accomplish. Another factor that could affect my results are the genres of music that are selected. Becuase it is the top 50 songs, there may be songs that are inherently slow. 

In [64]:
import requests
import pandas as pd
import base64
import json
import urllib

In [65]:
Client_ID = pd.read_csv('Spotify_Keys_9-19.txt')['Client_ID'].iloc[0]

In [66]:
#Client_ID

In [67]:
Client_Secret = pd.read_csv('Spotify_Keys_9-19.txt')['Client_Secret'].iloc[0]

In [68]:
#Client_Secret

In [69]:
client_cred = base64.b64encode(str(Client_ID + ":" + Client_Secret).encode("ascii"))

In [70]:
headers = {"Authorization" : "Basic {}".format(client_cred.decode("ascii"))}

In [71]:
payload = {'grant_type' : 'client_credentials'}
url = 'https://accounts.spotify.com/api/token'

In [72]:
session_key_response = requests.post(url = url, data = payload, headers = headers)

In [73]:
session_key_response.status_code

200

In [74]:
session_headers = {"Authorization" : "Bearer {}".format(session_key_response.json()['access_token'])}

In [75]:
search_url1 = 'https://api.spotify.com/v1/search?q={}&type={}&limit=50'.format(urllib.parse.quote('year:1969'),'track')

In [76]:
search_url1

'https://api.spotify.com/v1/search?q=year%3A1969&type=track&limit=50'

In [77]:
search_response1 = requests.get(url = search_url1, headers = session_headers)

In [78]:
search_response1.status_code

200

In [79]:
search_url2 = 'https://api.spotify.com/v1/search?q={}&type={}&limit=50'.format(urllib.parse.quote('year:2023'),'track')

In [80]:
search_response2 = requests.get(url = search_url2, headers = session_headers)

In [81]:
search_response2.status_code

200

In [82]:
search_data1 = search_response1.json()

In [83]:
search_data2 = search_response2.json()

In [84]:
search_df1 = pd.DataFrame(search_data1['tracks']['items'])

In [85]:
search_df2 = pd.DataFrame(search_data2['tracks']['items'])

In [116]:
#search_df1.head()

In [88]:
#search_df2.head()

In [89]:
audio_features_url1 = 'https://api.spotify.com/v1/audio-features?ids={}'.format(','.join(list(search_df1['id'])))

In [90]:
audio_features_url2 = 'https://api.spotify.com/v1/audio-features?ids={}'.format(','.join(list(search_df2['id'])))

In [91]:
features_response1 = requests.get(url = audio_features_url1, headers = session_headers)

In [92]:
features_response2 = requests.get(url = audio_features_url2, headers = session_headers)

In [93]:
features_old = features_response1.json()

In [94]:
features_new = features_response2.json()

In [95]:
oldfeatures_df = pd.DataFrame(features_old['audio_features'])

In [96]:
newfeatures_df = pd.DataFrame(features_new['audio_features'])

In [97]:
joined_dfold = pd.merge(search_df1, oldfeatures_df, how = 'inner', on = 'id')

In [98]:
joined_dfnew = pd.merge(search_df2, newfeatures_df, how = 'inner', on = 'id')

In [117]:
#joined_dfold.columns

In [106]:
finaldfold = joined_dfold.drop(columns=['available_markets', 'disc_number','type_y', 'uri_y', 'track_href','preview_url','is_local' ,'uri_x', 'analysis_url', 'duration_ms_y', 'duration_ms_x','external_ids', 'external_urls', 'href'])

In [107]:
finaldfnew = joined_dfnew.drop(columns=['available_markets', 'disc_number','type_y', 'uri_y', 'track_href','preview_url','is_local' ,'uri_x', 'analysis_url', 'duration_ms_y', 'duration_ms_x','external_ids', 'external_urls', 'href'])

In [108]:
#finaldfold.head()

,album,artists,explicit,id,name,popularity,track_number,type_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,4BP3uh0hFLFRb5cjsgLqDh,Fortunate Son,86,6,track,0.640,0.663,0,-7.516,1,0.0374,0.2010,0.008060,0.152,0.663,132.770,4
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,20OFwXhEXf12DzwXmaV7fj,Bad Moon Rising,83,5,track,0.508,0.774,2,-5.973,1,0.0321,0.0586,0.000005,0.063,0.942,178.946,4
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,6dGnYIeXmHdcikdzNNDMm2,Here Comes The Sun - Remastered 2009,85,7,track,0.557,0.540,9,-10.484,1,0.0347,0.0339,0.002480,0.179,0.394,129.171,4
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,62AuGbAkt8Ox2IrFFb8GKV,Sweet Caroline,79,13,track,0.529,0.127,11,-16.066,1,0.0274,0.6110,0.000109,0.237,0.578,63.050,4
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,6H3kDe7CGoWYBabAeVWGiD,Gimme Shelter,78,1,track,0.634,0.630,1,-8.277,0,0.0310,0.4470,0.039000,0.170,0.489,118.628,4


In [109]:
#finaldfnew.head()

,album,artists,explicit,id,name,popularity,track_number,type_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,4KULAymBBJcPRpk1yO4dOG,I Remember Everything (feat. Kacey Musgraves),93,11,track,0.429,0.453,0,-7.746,1,0.0459,0.55400,0.000002,0.1020,0.155,77.639,4
1,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,True,2IGMVunIBsBLtEQyoI1Mu7,Paint The Town Red,100,1,track,0.868,0.538,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,99.968,4
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,True,4rXLjWdF2ZZpXCVTfWcshS,fukumean,95,6,track,0.847,0.622,1,-6.747,0,0.0903,0.11900,0.000000,0.2850,0.220,130.001,4
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,True,1kuGVB7EU95pJObxwvfwKS,vampire,95,3,track,0.511,0.532,5,-5.745,1,0.0578,0.17700,0.000000,0.2910,0.350,138.005,4
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,True,3IX0yuEVvDbnqUwMBB3ouC,bad idea right?,93,2,track,0.627,0.879,9,-3.446,1,0.0955,0.00193,0.000007,0.0647,0.748,129.979,4


In [110]:
merged_frames = [finaldfold , finaldfnew]

In [111]:
result = pd.concat(merged_frames)

In [113]:
#result

In [115]:
result.sort_values(by = 'energy', ascending = False )

,album,artists,explicit,id,name,popularity,track_number,type_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
43,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,0yVs7eSL8mPnIu2CGKHpUQ,Communication Breakdown - 1990 Remaster,64,7,track,0.399,0.9460,2,-6.998,1,0.1640,0.05720,0.000007,0.7300,0.447,175.009,4
29,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,4BVE5tIIpYabZcYCbephNW,Touch Me,69,2,track,0.425,0.9030,1,-4.801,1,0.0732,0.40500,0.012800,0.1170,0.769,108.748,4
11,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,0hCB0YR03f6AmQaHbwWDe8,Whole Lotta Love - 1990 Remaster,77,1,track,0.412,0.9020,9,-11.600,1,0.4050,0.04840,0.131000,0.4050,0.422,89.740,4
32,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,True,42VsgItocQwOQC3XWZ8JNA,FE!N (feat. Playboi Carti),91,8,track,0.569,0.8820,3,-2.777,0,0.0600,0.03160,0.000000,0.1420,0.201,148.038,4
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,True,3IX0yuEVvDbnqUwMBB3ouC,bad idea right?,93,2,track,0.627,0.8790,9,-3.446,1,0.0955,0.00193,0.000007,0.0647,0.748,129.979,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,01SfTM5nfCou5gQL70r6gs,Golden Slumbers - Remastered 2009,68,14,track,0.419,0.1520,0,-12.126,1,0.0277,0.32600,0.003400,0.1130,0.239,80.608,4
35,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,4K1imZQQ0yKtJ40vGmUajS,Girl from the North Country,67,1,track,0.607,0.1420,7,-23.455,1,0.0382,0.84300,0.000197,0.1210,0.305,118.867,4
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,62AuGbAkt8Ox2IrFFb8GKV,Sweet Caroline,79,13,track,0.529,0.1270,11,-16.066,1,0.0274,0.61100,0.000109,0.2370,0.578,63.050,4
46,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,False,11VwZwNF29HrqwalYUMitb,Pale Blue Eyes,66,4,track,0.637,0.1070,5,-18.759,1,0.0349,0.91100,0.001460,0.1030,0.296,82.879,4


Looking at the final result of my collected data, without further analysis I can't fully confirm or deny my hypothesis. However when looking at the results of the final table the top three songs in terms of energy are from 1969. Even the song with the lowest energy is from 2023, but again without further analysis it would be hard to have a conclusive answer to the hypothesis being tested. One limitation when trying to draw conclusions is the fact the years of the songs are not provided. Going forward, next steps might be finding the average energy level of each year and also as I mentioned above getting other years to compare would help in verifying my conclusions.